In [1]:
import json
import re
import pandas as pd
from mplsoccer import Pitch, VerticalPitch
from adjustText import adjust_text
from highlight_text import ax_text
import matplotlib.pyplot as plt

from mplsoccer import Pitch, VerticalPitch
from adjustText import adjust_text
from highlight_text import ax_text
import matplotlib.pyplot as plt
from highlight_text import fig_text



In [2]:
def extract_json_from_html(html_path, save_output=False):
    html_file = open(html_path, 'r')
    html = html_file.read()
    html_file.close()
    regex_pattern = r'(?<=require\.config\.params\["args"\].=.)[\s\S]*?;'
    data_txt = re.findall(regex_pattern, html)[0]

    # add quotations for json parser
    data_txt = data_txt.replace('matchId', '"matchId"')
    data_txt = data_txt.replace('matchCentreData', '"matchCentreData"')
    data_txt = data_txt.replace('matchCentreEventTypeJson', '"matchCentreEventTypeJson"')
    data_txt = data_txt.replace('formationIdNameMappings', '"formationIdNameMappings"')
    data_txt = data_txt.replace('};', '}')

    if save_output:
        # save json data to txt
        output_file = open(f"{html_path}.txt", "wt")
        n = output_file.write(data_txt)
        output_file.close()

    return data_txt




def extract_data_from_dict(data):
    # load data from json
    event_types_json = data["matchCentreEventTypeJson"]
    formation_mappings = data["formationIdNameMappings"]
    events_dict = data["matchCentreData"]["events"]
    teams_dict = {data["matchCentreData"]['home']['teamId']: data["matchCentreData"]['home']['name'],
                  data["matchCentreData"]['away']['teamId']: data["matchCentreData"]['away']['name']}
    players_dict = data["matchCentreData"]["playerIdNameDictionary"]
    # create players dataframe
    players_home_df = pd.DataFrame(data["matchCentreData"]['home']['players'])
    players_home_df["teamId"] = data["matchCentreData"]['home']['teamId']
    players_away_df = pd.DataFrame(data["matchCentreData"]['away']['players'])
    players_away_df["teamId"] = data["matchCentreData"]['away']['teamId']
    players_df = pd.concat([players_home_df, players_away_df])
    players_ids = data["matchCentreData"]["playerIdNameDictionary"]
    return events_dict, players_df, teams_dict



match_html_path = '/Users/aritramajumdar/Downloads/Whoscored/Worldcup/Worldcup_groupstage/Spain 1-1 Germany - FIFA World Cup 2022 Live.html'





json_data_txt = extract_json_from_html(match_html_path)
data = json.loads(json_data_txt)
events_dict, players_df, teams_dict = extract_data_from_dict(data)
df_events = pd.DataFrame(events_dict)


df_events.head()

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,endY,relatedEventId,relatedPlayerId,blockedX,blockedY,goalMouthZ,goalMouthY,isShot,cardType,isGoal
0,2.488098e+09,2,0,0.0,336,0.0,0.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 32, 'displayName': 'Start'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.488098e+09,2,0,0.0,338,0.0,0.0,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 32, 'displayName': 'Start'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.488098e+09,3,0,0.0,338,50.1,50.1,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,60.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.488098e+09,4,0,2.0,338,29.6,61.2,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,33.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.488098e+09,5,0,5.0,338,31.6,33.4,0,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 1, 'displayName': 'Pass'}",...,93.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_events['qualifiers'][3]

[{'type': {'value': 213, 'displayName': 'Angle'}, 'value': '4.8'},
 {'type': {'value': 140, 'displayName': 'PassEndX'}, 'value': '31.3'},
 {'type': {'value': 56, 'displayName': 'Zone'}, 'value': 'Back'},
 {'type': {'value': 212, 'displayName': 'Length'}, 'value': '18.8'},
 {'type': {'value': 178, 'displayName': 'StandingSave'}},
 {'type': {'value': 141, 'displayName': 'PassEndY'}, 'value': '33.7'}]

In [4]:
items = []

new_df = pd.DataFrame()

for i in range(0, len(df_events)):
    list_of_all_values5 = [value for elem in df_events['qualifiers'][i] for value in elem.values()]
    try:
        if {'value': 20, 'displayName': 'RightFoot'} in list_of_all_values5:
            items.append("Right Foot") 
        elif {'value': 72, 'displayName': 'LeftFoot'} in list_of_all_values5:
            items.append("Left Foot")
        elif {'value': 15, 'displayName': 'Head'} in list_of_all_values5:
            items.append("Head")
        elif {'value': 21, 'displayName': 'OtherBodyPart'} in list_of_all_values5:
            items.append("Other")
            
        else:
            items.append(0)
         
    except:
        items.append(0)

        
new_df['shot_body_part_name'] = items
frames = [df_events, new_df]
data = pd.concat(frames, axis = 1)
shots = data.loc[data['isShot'] == True]

In [5]:
shots.head()

,id,eventId,minute,second,teamId,x,y,expandedMinute,period,type,...,relatedEventId,relatedPlayerId,blockedX,blockedY,goalMouthZ,goalMouthY,isShot,cardType,isGoal,shot_body_part_name
139,2.488099e+09,102,6,25.0,338,79.9,70.3,6,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 15, 'displayName': 'SavedShot'}",...,101.0,137467.0,98.0,52.5,32.3,47.5,True,NaN,NaN,Right Foot
431,2.488100e+09,299,21,13.0,338,77.1,73.6,21,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 13, 'displayName': 'MissedShots'}",...,298.0,44721.0,NaN,NaN,9.7,57.0,True,NaN,NaN,Right Foot
479,2.488101e+09,194,24,11.0,336,83.8,32.2,24,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 13, 'displayName': 'MissedShots'}",...,NaN,NaN,NaN,NaN,1.4,63.7,True,NaN,NaN,Left Foot
496,2.488101e+09,337,25,58.0,338,85.2,42.3,25,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 15, 'displayName': 'SavedShot'}",...,336.0,261212.0,86.8,43.0,19.0,48.3,True,NaN,NaN,Left Foot
575,2.488101e+09,256,29,39.0,336,72.1,66.2,29,"{'value': 1, 'displayName': 'FirstHalf'}","{'value': 13, 'displayName': 'MissedShots'}",...,255.0,395252.0,NaN,NaN,94.4,36.6,True,NaN,NaN,Right Foot


In [6]:
df_events.shape

(1650, 26)

In [7]:
def next_element(lst, element):
    idx = lst.index(element)
    return lst[idx +1]

items1 = []
new_df1 = pd.DataFrame()
for i in range(0, len(df_events)):
    list_of_all_values = [value for elem in df_events['qualifiers'][i] for value in elem.values()]
    try:
        items1.append(float(next_element(list_of_all_values, {'value': 213, 'displayName': 'Angle'})))
    except:
        items1.append(0)
    

In [8]:
new_df1['Angle'] = items1
print(new_df1)

      Angle
0       0.0
1       0.0
2       2.8
3       4.8
4       0.9
...     ...
1645    0.0
1646    0.0
1647    0.0
1648    0.0
1649    0.0

[1650 rows x 1 columns]


In [9]:
df_events['qualifiers']

0                                                      []
1                                                      []
2       [{'type': {'value': 56, 'displayName': 'Zone'}...
3       [{'type': {'value': 213, 'displayName': 'Angle...
4       [{'type': {'value': 155, 'displayName': 'Chipp...
                              ...                        
1645                                                   []
1646                                                   []
1647                                                   []
1648    [{'type': {'value': 131, 'displayName': 'TeamP...
1649    [{'type': {'value': 59, 'displayName': 'Jersey...
Name: qualifiers, Length: 1650, dtype: object